In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/My Drive
%cd Weather

In [ ]:
! ls

### 1 Install Meteostat
 * WEB:https://meteostat.net/en/place/CA-QVT9?t=2021-09-13/2021-09-19
 * Script sample: https://pypi.org/project/meteostat/
 * Developer: https://github.com/sponsors/clampr

In [ ]:
#! pip install meteostat
import sys
!{sys.executable} -m pip install --upgrade meteostat

In [ ]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
from geopy.geocoders import Nominatim

# Set time period
start = datetime(1930, 9, 15)
#end = datetime(2021, 9, 15)
end = datetime.now()
# e.g. Create Point for Vancouver, BC
#location = Point(49.2497, -123.1193, 70)

# Set location
location_name='Berlin'

# get Latitude and Longitude
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode(location_name)
latitude_value=location.latitude
longitude_value=location.longitude
print (location_name,'  :',latitude_value, longitude_value,)

# TB: Create Point for Berlin, Germany
location = Point(latitude_value, longitude_value, 70)

# Get daily data
data = Daily(location, start, end)
data = data.fetch()

#TB save as csv
data.to_csv(location_name+'.csv')

# create a horizontal plot
def figureplots(rownew, columnew, datanew, colornew, labelnew, dstartTBnew, dnowTBnew):
  #graphs in a figures using subplots
  axs[columnew, rownew].plot(datanew, color=colornew)
  axs[columnew, rownew].set_xlim(dstartTBnew, dnowTBnew)
  axs[columnew, rownew].set_xlabel('days')
  axs[columnew, rownew].set_ylabel(labelnew)


#define start and end date
dnowTB = datetime.now()
dstartTB=datetime(2021,1,1)

fig, axs = plt.subplots(2, 2, figsize=(15, 8))
plt.rc('font', size=15)
#figureplots(rownew, columnew, datanew, colornew, labelnew, dstartTB, dnowTB)
figureplots(0, 0, data['tavg'], 'red', 'Average Temperature', dstartTB, dnowTB)
figureplots(1, 0, data['tmin'], 'green', 'Minimum Temperature', dstartTB, dnowTB)
figureplots(0, 1, data['tmax'], 'orange', 'Max Temperature', dstartTB, dnowTB)
figureplots(1, 1, data['tavg'], 'blue', 'All data: Average Temperature', start, dnowTB)
plt.show()



In [ ]:
#view of dataframe
data.tail()

# interpolation using spline

In [ ]:
#from sklearn.datasets import load_boston
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
datacopied=data
datacopied.tail()

In [ ]:
from scipy import interpolate as itp
y = datacopied['tavg'][20000:len(datacopied)]

x = range(0, len(y))
print(x,len(y))
mytck,myu=itp.splprep([x,y])
xnew,ynew= itp.splev(np.linspace(0,1,len(y)*2),mytck)
plt.figure(figsize=(12, 6))
plt.plot(xnew,ynew,'+',c="r")

plt.figure(figsize=(12, 6))
plt.plot(x, y, '.', c="b")
plt.grid()
plt.show()


knot_numbers = 10000
x_new = np.linspace(0, 1, knot_numbers+2)[1:-1]
q_knots = np.quantile(x, x_new)


t,c,k = interpolate.splrep(x, y, t=q_knots, s=1)
yfit = interpolate.BSpline(t,c,k)(x)


plt.figure(figsize=(8, 4))
plt.title("Spline curve fitting")
plt.plot(x, y, '.', c="y", label="original")
plt.plot(x, yfit, '-', c="r", label="spline fit")
plt.legend(loc='best', fancybox=True, shadow=True)

plt.grid()
plt.show()

def spline(knots, y):
    x = range(0, len(y))
    x_new = np.linspace(0, 1, knots+2)[1:-1]
    q_knots = np.quantile(x, x_new)
    t, c, k = interpolate.splrep(x, y, t=q_knots, s=3)
    yfit = interpolate.BSpline(t,c, k)(x)
    return yfit


knots = [1, 10, 20, 130]
i = 0

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 5))



for row in range(2):
    for col in range(2):
        #ax[row][col].plot(x, y, '.',c="g", markersize=2)
        yfit = spline(knots[i], y)
        ax[row][col].plot(x, yfit, 'r')
        ax[row][col].set_title("Knots number - "+str(knots[i]))
        ax[row][col].grid()
        i=i+1

plt.tight_layout()
plt.show()

# Plotly graph and gap detections:

In [ ]:
datacopied

In [47]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import interpolate as itp
from sklearn.linear_model import LinearRegression
# Assuming datacopied is already loaded, e.g., datacopied = pd.read_csv('your_file.csv', parse_dates=['time'], index_col='time')
# Ensure 'time' is the index and in datetime format
datacopied.index = pd.to_datetime(datacopied.index)
# Filter data to start from 1946
filtered_data = datacopied[datacopied.index >= '1946-01-01']
# Further filter out rows where 'tavg' is NaN or zero
filtered_data = filtered_data[filtered_data['tavg'].notna() & (filtered_data['tavg'] != 0)]
# Identify the gaps
gap_threshold = np.timedelta64(10, 'D')  # Define what constitutes a gap (e.g., 10 days)
time_diffs = filtered_data.index.to_series().diff().fillna(pd.Timedelta(seconds=0))
gap_indices = np.where(time_diffs > gap_threshold)[0]
# Function to perform interpolation and plotting
def plot_segment(fig, start_idx, end_idx):
    segment = filtered_data.iloc[start_idx:end_idx]
    x = segment.index
    y = segment['tavg']

    # Convert time values to ordinal numbers for spline interpolation
    x_ordinal = x.map(pd.Timestamp.toordinal).values

    # Check if there are enough data points for interpolation
    if len(x_ordinal) > 3:
        # Interpolation
        mytck, myu = itp.splprep([x_ordinal, y], s=0)  # s=0 for exact fit
        xnew, ynew = itp.splev(np.linspace(0, 1, 15000), mytck)
        # Convert ordinal numbers back to datetime for x-axis
        xnew_datetime = [pd.Timestamp.fromordinal(int(val)) for val in xnew]
    else:
        # If not enough points, plot the data directly
        xnew_datetime = x
        ynew = y

    # Add the segment to the figure
    fig.add_trace(go.Scatter(x=xnew_datetime, y=ynew, mode='lines'))
# Create a Plotly figure
fig = go.Figure()
if len(gap_indices) == 0:
    # No gaps detected
    plot_segment(fig, 0, len(filtered_data))
else:
    start_idx = 0
    for gap_idx in gap_indices:
        end_idx = gap_idx
        plot_segment(fig, start_idx, end_idx)
        start_idx = end_idx
    # Plot the last segment
    plot_segment(fig, start_idx, len(filtered_data))
# Calculate the linear trend
x_ordinal = filtered_data.index.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
y = filtered_data['tavg'].values
model = LinearRegression().fit(x_ordinal, y)
trend_line = model.predict(x_ordinal)
# Add the linear trend to the plot
fig.add_trace(go.Scatter(
    x=filtered_data.index,
    y=trend_line,
    mode='lines',
    line=dict(color='green', width=2),
    name='Trend Line'
))
# Update figure layout
fig.update_layout(
    title='Time vs tavg with Linear Trend',
    xaxis_title='Time',
    yaxis_title='tavg',
    xaxis=dict(tickangle=45)
)
# Show the plot
fig.show()